In [2]:
import pandas as pd

def generate_layerinfo(mapping_path: str,
                       output_path: str = "layerinfo.csv",
                       default_layer_num: int = 1) -> pd.DataFrame:
    """
    metabolite_mapping.set (tab-separated) 파일을 읽어
    pathway 코드별로 metabolite 개수를 세고,
    모두 같은 default_layer_num 값을 부여한 뒤
    layerinfo.csv 로 저장합니다.

    Parameters
    ----------
    mapping_path : str
        pathway-to-metabolite 매핑 파일 경로 (예: "metabolite_mapping.set").
    output_path : str, optional
        생성할 CSV 파일명, by default "layerinfo.csv".
    default_layer_num : int, optional
        pathway마다 부여할 레이어 수, by default 1.

    Returns
    -------
    pd.DataFrame
        생성된 DataFrame (columns: pathway, node_num, layer_num).
    """
    # 1) 매핑 파일 읽기
    df = pd.read_csv(mapping_path,
                     sep='\t',
                     header=None,
                     names=["pathway", "metabolite"],
                     dtype={"pathway": str, "metabolite": str})

    # 2) pathway별 metabolite 개수 집계
    agg = (
        df
        .groupby("pathway")
        .size()
        .reset_index(name="node_num")
    )

    # 3) layer_num 컬럼 추가
    agg["layer_num"] = default_layer_num

    # 4) CSV 저장
    agg.to_csv(output_path, index=False)

    return agg


filepath = 'metabolite_mapping.set'
layerinfo = generate_layerinfo(filepath, output_path='layerinfo.csv', default_layer_num=6)